In [119]:
from IPython.display import display, HTML, Image, clear_output
import time
import random
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from jupyter_ui_poll import ui_events

In [120]:
data_consent_info = """
DATA CONSENT INFORMATION:

Please read:
We wish to record your response data
to an anonymised public data repository.
Your data will be used for educational teaching purposes
practising data analysis and visualisation.
Please type yes in the box below if you consent to the upload."""

id_instructions = """Enter your anonymised ID

To generate an anonymous 4-letter unique user identifier please enter:
- two letters based on the initials (first and last name) of a childhood friend
- two letters based on the initials (first and last name) of a favourite actor
e.g. if your friend was called Charlie Brown and film star was Tom Cruise
then your unique identifer would be CBTC"""

formurl = "https://docs.google.com/forms/d/e/1FAIpQLSf95hp02hJIl9GrhBLiKFHXLepEwSclrQj-jmN6ljF9htIL8g/viewform?usp=sf_link"

header_text = "Welcome to the Appropriate Number Sense (ANS) Test"
intro_text = """Once you have agreed to the data consent information below, the test will start.
2 images will appear on your screen, both will have a set number of dots in the image.
Some images will have blue and yellow dots, others will have both sets of dots be green.
You will be able to see this image for 0.75 seconds, after which the image will disappear. 
Below the image you will see two buttons: one labelled "Left" and one labelled "Right"

The rules for the test are as follows:

Please click the button for whichever image had MORE dots.
You will have up to 3 seconds to make a selection, after which a new image will appear.
If you click the button before the 3 seconds, the next image will immediately load, so there's no need to wait.
There are a total of 128 questions, we estimate the test will take about 2 minutes to complete.
You will not be able to see your results at the end of the test."""

style1 = 'color: black; font-size: 40px;'

htmlOut = HTML(f"<span style ='{style1}'>{header_text}</span>")

In [121]:
#list-making functions
def make64 (list):
    longerlist = list
    for i in range(2):
        for j in range(len(list)):
            longerlist.append(list[j])

    return longerlist

def ratList (left, right):
    rat = []
    for i in range(len(left)):
        if left[i] < right[i]:
            rat.append(left[i]/right[i])
        else:
            rat.append(right[i]/left[i])

    return rat

def typeList (type):
    t = []
    for i in range(64):
        t.append(type)

    return t

In [122]:
#necessary variables cell

pic1 = Image("10vs9.png", width = 500)
pic2 = Image("12vs14.png", width = 500)
pic3 = Image("12vs16.png", width = 500)
pic4 = Image("12vs9.png", width = 500)
pic5 = Image("14vs12.png", width = 500)
pic6 = Image("15vs20.png", width = 500)
pic7 = Image("16vs12.png", width = 500)
pic8 = Image("16vs18.png", width = 500)
pic9 = Image("18vs16.png", width = 500)
pic10 = Image("18vs20.png", width = 500)
pic11 = Image("18vs21.png", width = 500)
pic12 = Image("20vs15.png", width = 500)
pic13 = Image("20vs18.png", width = 500)
pic14 = Image("21vs18.png", width = 500)
pic15 = Image("9vs10.png", width = 500)
pic16 = Image("9vs12.png", width = 500)

pic1a = Image("G10vs9.png", width = 500)
pic2a = Image("G12vs14.png", width = 500)
pic3a = Image("G12vs16.png", width = 500)
pic4a = Image("G12vs9.png", width = 500)
pic5a = Image("G14vs12.png", width = 500)
pic6a = Image("G15vs20.png", width = 500)
pic7a = Image("G16vs12.png", width = 500)
pic8a = Image("G16vs18.png", width = 500)
pic9a = Image("G18vs16.png", width = 500)
pic10a = Image("G18vs20.png", width = 500)
pic11a = Image("G18vs21.png", width = 500)
pic12a = Image("G20vs15.png", width = 500)
pic13a = Image("G20vs18.png", width = 500)
pic14a = Image("G21vs18.png", width = 500)
pic15a = Image("G9vs10.png", width = 500)
pic16a = Image("G9vs12.png", width = 500)

piclist = [pic1, pic2, pic3, pic4, pic5, pic6, pic7, pic8, pic9, pic10, pic11, pic12,
          pic13, pic14, pic15, pic16]
filename = ["10vs9.png", "12vs14.png", "12vs16.png", "12vs9.png", "14vs12.png", "15vs20.png", "16vs12.png", "16vs18.png",
            "18vs16.png", "18vs20.png", "18vs21.png", "20vs15.png", "20vs18.png", "21vs18.png", "9vs10.png", "9vs12.png"]
left = [10, 12, 12, 12, 14, 15, 16, 16, 18, 18, 18, 20, 20, 21, 9, 9]
right = [9, 14, 16, 9, 12, 10, 12, 18, 16, 20, 21, 15, 18, 18, 10, 12]

piclistA = [pic1a, pic2a, pic3a, pic4a, pic5a, pic6a, pic7a, pic8a, pic9a, pic10a, pic11a, pic12a,
          pic13a, pic14a, pic15a, pic16a]
filenameA = ["G10vs9.png", "G12vs14.png", "G12vs16.png", "G12vs9.png", "G14vs12.png", "G15vs20.png", "G16vs12.png", "G16vs18.png",
            "G18vs16.png", "G18vs20.png", "G18vs21.png", "G20vs15.png", "G20vs18.png", "G21vs18.png", "G9vs10.png", "G9vs12.png"]
rat = ratList(left, right)
sList = typeList('standard')
gList = typeList('green')


#expands the list to include 64 values in each - intended test size
piclist = make64(piclist)
filename = make64(filename)
piclistA = make64(piclistA)
filenameA = make64(filenameA)
leftlist = make64(left)
rightlist = make64(right)
ratio = make64(rat)

#combines lists in correct corresponding order to make one large test set with the mixed question types
piclist = piclist + piclistA
filename = filename + filenameA
leftlist = leftlist +leftlist.copy()
rightlist = rightlist + rightlist.copy()
ratio = ratio + ratio.copy()
typelist = sList + gList

#randomly shuffles all lists but in the same order
random.seed(1)
random.shuffle(piclist)
random.seed(1)
random.shuffle(filename)
random.seed(1)
random.shuffle(leftlist)
random.seed(1)
random.shuffle(rightlist)
random.seed(1)
random.shuffle(ratio)
random.seed(1)
random.shuffle(typelist)

print(len(typelist))

128


In [123]:
# dictionaries to be used
#for the results storing
resultsdict = {
    'filename' : filename,
    'nl' : leftlist,
    'nr' : rightlist,
    'ratio' :ratio,
    'type' : typelist,
    'ans' : [],
    'correct': [],
    'react' : []}

#for the buttons
event_info = {
    'type': '',
    'description': '',
    'time': -1
}

In [124]:
#button functions
def register_btn_event(btn):
    event_info['type'] = "button click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return 

def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):
    start_wait = time.time()

    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

In [125]:
#function for a single test
def ANSsingle (images, leftlist, rightlist, main_area, btn1, btn2):    
    tally = 0

    with main_area: display(images)
    time.sleep(0.75)
    with main_area: clear_output()

    start = time.time()

    btn1.on_click(register_btn_event)
    btn2.on_click(register_btn_event)

    result = wait_for_event(timeout=3)
    
    end = time.time()
    reaction = end - start
    
    if leftlist > rightlist:
        anstrue = 'Left'
    else:
        anstrue = 'Right'
        
    if  result['description'] == anstrue and reaction < 3:
        tally += 1
        resultsdict['correct'].append(1)
        resultsdict['ans'].append(anstrue)
        resultsdict['react'].append(reaction)
    else:
        resultsdict['correct'].append(0)
        resultsdict['ans'].append(anstrue)
        resultsdict['react'].append(reaction)

    return tally, reaction

In [126]:
#function that repeats each individual test and records results
def ANSblock (images, leftlist, rightlist):
    resulttotal = 0
    reacttotal = 0

    top_area = widgets.Output(layout={"height":"60px"})
    main_area = widgets.Output(layout={"height":"300px"})
    bottom_area = widgets.Output(layout={"height":"60px"})

    btn1 = widgets.Button(description="Left", layout={"height":"60px", "width":"250px"})
    btn2 = widgets.Button(description="Right", layout={"height":"60px", "width":"250px"})
    panel = widgets.HBox([btn1, btn2])
   
    top_area.append_display_data(HTML("<h1>Left or Right</h1>"))
    bottom_area.append_display_data(panel)

    display(top_area)
    display(main_area)
    display(bottom_area)
    
    for i in range(len(images)):
        y = ANSsingle(images[i], leftlist[i], rightlist[i], main_area, btn1, btn2)
        resulttotal += y[0]
        reacttotal += y[1]

    clearui(top_area, main_area, bottom_area)
        
    return resulttotal, reacttotal

In [127]:
#functions to reset test conditions
def cleardict():
    resultsdict['correct'] = []
    resultsdict['ans'] = []
    resultsdict['react'] = []
    return

def clearui(top, main, bot):
    with top: clear_output()
    with main: clear_output()
    with bot: clear_output()
    return

In [128]:
#function to send the results to the google form linked in the formurl variable
def send_to_google_form(bigdict, formurl):
    form_id = formurl[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in bigdict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = bigdict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

In [129]:
def bigdict (result, time, id):
    resultsdf = pd.DataFrame(resultsdict)
    resultsjson = resultsdf.to_json()

    finaldict = {
        'id' : id,
        'score' : result,
        'time': time,
        'resultsjson' : resultsjson}

    return finaldict

In [130]:
cleardict()
display(htmlOut)
time.sleep(1)
print(intro_text)
time.sleep(3)

print(data_consent_info)
result = input("> ")
time.sleep(1)

if result == "yes":
    print("Thanks for your participation.")
    print("Please contact philip.lewis@ucl.ac.uk")
    print("If you have any questions or concerns")
    print("regarding the stored results.")
else:
    # end code execution by raising an exception
    raise(Exception("User did not consent to continue test."))

print(id_instructions)
user_id = input("> ").upper()
print("User entered id:", user_id)
time.sleep(1)

input("When you are ready, press ENTER on your keyboard to begin the test")
time.sleep(2)

x = ANSblock(piclist, leftlist, rightlist)

result = x[0]
time = x[1]

finaldict = bigdict(result, time, user_id)
send_to_google_form(finaldict, formurl)

print("Thank you for your participation.")

Once you have agreed to the data consent information below, the test will start.
2 images will appear on your screen, both will have a set number of dots in the image.
Some images will have blue and yellow dots, others will have both sets of dots be green.
You will be able to see this image for 0.75 seconds, after which the image will disappear. 
Below the image you will see two buttons: one labelled "Left" and one labelled "Right"

The rules for the test are as follows:

Please click the button for whichever image had MORE dots.
You will have up to 3 seconds to make a selection, after which a new image will appear.
If you click the button before the 3 seconds, the next image will immediately load, so there's no need to wait.
There are a total of 128 questions, we estimate the test will take about 2 minutes to complete.
You will not be able to see your results at the end of the test.

DATA CONSENT INFORMATION:

Please read:
We wish to record your response data
to an anonymised public d

>  yes


Thanks for your participation.
Please contact philip.lewis@ucl.ac.uk
If you have any questions or concerns
regarding the stored results.
Enter your anonymised ID

To generate an anonymous 4-letter unique user identifier please enter:
- two letters based on the initials (first and last name) of a childhood friend
- two letters based on the initials (first and last name) of a favourite actor
e.g. if your friend was called Charlie Brown and film star was Tom Cruise
then your unique identifer would be CBTC


>  tezt


User entered id: TEZT


When you are ready, press ENTER on your keyboard to begin the test 


Output(layout=Layout(height='60px'), outputs=({'output_type': 'display_data', 'data': {'text/plain': '<IPython…

Output(layout=Layout(height='300px'))

Output(layout=Layout(height='60px'), outputs=({'output_type': 'display_data', 'data': {'text/plain': "HBox(chi…

Thank you for your participation.


In [131]:
print(resultsdict)

{'filename': ['9vs10.png', '9vs10.png', 'G16vs18.png', '14vs12.png', 'G9vs12.png', 'G12vs14.png', 'G14vs12.png', '12vs14.png', '14vs12.png', '12vs9.png', 'G12vs9.png', 'G20vs18.png', 'G9vs10.png', 'G18vs21.png', 'G16vs18.png', 'G10vs9.png', '16vs12.png', 'G20vs18.png', '16vs18.png', '10vs9.png', '15vs20.png', '18vs20.png', '14vs12.png', 'G9vs12.png', 'G18vs20.png', '21vs18.png', 'G9vs10.png', 'G16vs12.png', '16vs12.png', '18vs20.png', '16vs18.png', '14vs12.png', 'G18vs16.png', 'G18vs21.png', '12vs16.png', 'G20vs15.png', '12vs14.png', '18vs21.png', '16vs12.png', '20vs15.png', 'G10vs9.png', '15vs20.png', '21vs18.png', '20vs15.png', '12vs9.png', 'G15vs20.png', '20vs15.png', 'G12vs16.png', 'G10vs9.png', 'G21vs18.png', '9vs12.png', 'G18vs21.png', '9vs10.png', 'G9vs10.png', 'G14vs12.png', '18vs20.png', 'G18vs20.png', 'G12vs14.png', '9vs12.png', 'G12vs14.png', 'G14vs12.png', '12vs16.png', '12vs9.png', '18vs16.png', 'G15vs20.png', 'G10vs9.png', '18vs21.png', 'G18vs16.png', 'G18vs20.png', '16vs